In [1]:
import http.client
import json
import os
from dotenv import load_dotenv
import pandas as pd
import re
from IPython.display import display, Markdown
from nltk import tokenize
from langchain.text_splitter import RecursiveCharacterTextSplitter
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from openai import OpenAI
#from background_ufunc import *
#import PyPDF2
import heapq
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document

In [ ]:
import os

# Create output directory
os.makedirs('results/QA_accuracy', exist_ok=True)

In [ ]:
credential = InteractiveBrowserCredential()
 
scope = [""]
 
token = credential.get_token(*scope)
 
print(token)
 
auth_token = token.token
'''
liteLLM_model = "llama-3.2"
client = OpenAI(base_url='https://llama-inference.stage.att.com', api_key=auth_token)
'''

liteLLM_model="gpt-4o-mini"
 
client = OpenAI(base_url='https://cdo-inference.stage.att.com', api_key=auth_token)
 


In [3]:


def question_finder(file_name):
    with open(file_name, 'r') as file:
        data = json.load(file)
    liteLLM_model="gpt-4o-mini"
    total_entries = len(data)
    yes_count = 0

    for entry in data:
        question = entry['question']
        ground_truth = entry['ground_truth_answer']
        generated = entry['generated_answer']
        
        result = check_answer_similarity(
            question=question,
            ground_truth=ground_truth,
            generated=generated,
            LLM=liteLLM_model
        )
        
        if "yes" in result.lower():
            yes_count += 1
        else:
            print(f"Question: {question}\nGround Truth: {ground_truth}\nGenerated: {generated}\nResult: {result}\n")

    proportion_yes = yes_count / total_entries
    print(f"Proportion of 'yes' responses: {proportion_yes}")
    return proportion_yes,file_name

#question_finder("results/public/1nn_lm_weird_word_replacement/evaluation_combined_1nn_lm_finetuned_embedding_weird_word_perturbed_generated_answers_threshold_0.1.json")


In [ ]:
client.models.list()

In [ ]:
import json
import os
import csv
"""
FILE: /Users/ag997x/Library/CloudStorage/OneDrive-AT&TServices,Inc/Desktop/att/results/compare results.ipynb
This script provides functionality to evaluate the semantic similarity between answers to a given question using a language model (LLM). 
It includes a function to compare answers and determine if they are semantically equivalent, as well as a utility to analyze a dataset 
of questions and answers stored in a JSON file.
Functions:
----------
1. check_answer_similarity(question, ground_truth, generated, LLM, tokens=10):
    - Compares the semantic meaning of two answers to a given question.
    - Parameters:
        - question (str): The question being analyzed.
        - ground_truth (str): The reference answer (ground truth).
        - generated (str): The generated answer to compare.
        - LLM (object): The language model to use for comparison.
        - tokens (int, optional): The maximum number of tokens for the LLM response. Default is 10.
    - Returns:
        - str: 'yes' if the answers are semantically equivalent, 'no' otherwise.
2. analyze_similarity(file_name):
    - Analyzes a JSON file containing questions, ground truth answers, and generated answers to calculate the proportion of semantically 
      equivalent answers.
    - Parameters:
        - file_name (str): The path to the JSON file containing the data.
    - Returns:
        - float: The proportion of 'yes' responses indicating semantic equivalence.
Usage:
------
- The script reads a JSON file containing a list of questions, ground truth answers, and generated answers.
- It uses the `check_answer_similarity` function to evaluate each entry and calculates the proportion of semantically equivalent answers.
- Example usage is provided at the end of the script.
Dependencies:
-------------
- Requires the `json` module for reading and parsing JSON files.
- Assumes the presence of a language model client (`liteLLM_model`) for generating text completions.
Note:
-----
- Ensure the JSON file is formatted correctly with keys: 'question', 'ground_truth_answer', and 'generated_answer'.
- The language model client (`liteLLM_model`) must be properly initialized before running the script.
"""


def check_answer_similarity(question, ground_truth, generated, LLM, tokens=10):
    def text_complete(prompt, LLM):
        text_completion = client.completions.create(
            prompt=prompt,
            model=LLM,
            max_tokens=tokens
        )
        text_completion.to_dict()
        return text_completion.choices[0].text.strip()

    prompt = f"""You are an expert at comparing answers to a given question. 
              Your task is to determine if the second answer is correct given that the first answer is the ground truth response, even if phrased differently. 
              The question is: '{question}'. 
              The first answer is: '{ground_truth}'. 
              The second answer is: '{generated}'. 
              Carefully analyze the semantic meaning of both answers and respond with 'yes' if the second answer is correct as measured 
              against the first with respect to the question, or 'no' if not. Even if part of second answer relevant to the question is incorrect, 
                 respond with 'no'. """

    result = text_complete(prompt, LLM)
    return result
'''
response = check_answer_similarity(
    question="What city served as Maria Garcia Alvarez's birthplace and what careers do her parents pursue?",
    ground_truth="Maria Garcia Alvarez was born in Madrid, Spain. Her father is a midwife and her mother is a civil engineer.",
    generated="I apologize for the confusion, but I cannot answer that question directly as I don't have access to specific information about Maria Garcia Alvarez's personal background.",
    LLM=liteLLM_model
)
# Load the JSON file
with open('data.json', 'r') as file:
    data = json.load(file)

# Initialize counters
total_entries = len(data)
yes_count = 0

# Iterate through each entry in the JSON file
for entry in data:
    question = entry['question']
    ground_truth = entry['ground_truth_answer']
    generated = entry['generated_answer']
    
    # Use the function to check similarity
    result = check_answer_similarity(
        question=question,
        ground_truth=ground_truth,
        generated=generated,
        LLM=liteLLM_model
    )
    
    # Count 'yes' responses
    if result.lower() == 'yes':
        yes_count += 1

# Calculate the proportion of 'yes' responses
proportion_yes = yes_count / total_entries
print(f"Proportion of 'yes' responses: {proportion_yes}")

print(response)
'''
def analyze_similarity(file_name):
    with open(file_name, 'r') as file:
        data = json.load(file)
    liteLLM_model="gpt-4o-mini"
    total_entries = len(data)
    yes_count = 0

    for entry in data:
        question = entry['question']
        ground_truth = entry['ground_truth_answer']
        generated = entry['generated_answer']
        
        result = check_answer_similarity(
            question=question,
            ground_truth=ground_truth,
            generated=generated,
            LLM=liteLLM_model
        )
        
        if "yes" in result.lower():
            yes_count += 1

    proportion_yes = yes_count / total_entries
    print(f"Proportion of 'yes' responses: {proportion_yes}")
    return proportion_yes,file_name

# Example usage
#proportion,file_name = analyze_similarity('data.json')


# Initialize a list to store results
results = []
import concurrent.futures

def process_j(j):
    local_results = []
    i = 0 + 1000 * j
    for root, dirs, files in os.walk('./results'):
        for file in files:
            if file.startswith('evaluation') and file.endswith('.json'):
                i += 1
                file_path = os.path.join(root, file)
                proportion, file_name = analyze_similarity(file_path)
                local_results.append({'file_name': file_name, 'proportion': proportion})
                print("file" + str(i))
    output_csv = f'results/QA_accuracy/comparison_results_{j}.csv'
    with open(output_csv, mode='w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['file_name', 'proportion'])
        writer.writeheader()
        writer.writerows(local_results)
    print(f"Results have been saved to {output_csv}")
    return local_results

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_j, j) for j in range(4)]
    for future in concurrent.futures.as_completed(futures):
        results.extend(future.result())

In [ ]:
c=0
for root, dirs, files in os.walk('./results'):
    for file in files:
        if file.startswith('evaluation') and file.endswith('.json'):
            file_path = os.path.join(root, file)
            print(file_path)
            c+=1
print(c)